In [264]:
#Objetive of this notebook is to generate two columns in dataset temperature and precipitation
#With the following structure: country, date, temp, prcp
#Google Big query and Google Maps Platform in order to retrieve countries for each location in each station retrieved

In [293]:
import numpy as np
import pandas as pd

In [294]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/lucas_botella/Downloads/tfm-covid-forecast-lb-278ff92b9d29.json"

In [295]:
# Set your own project id here
PROJECT_ID = 'poetic-bison-304616'
from google.cloud import bigquery
client = bigquery.Client(project=PROJECT_ID)

In [626]:
table1_stations = bigquery.TableReference.from_string(
    "bigquery-public-data.noaa_gsod.stations"
)
dataframe_stations = client.list_rows(
    table1_stations,
    selected_fields=[
        bigquery.SchemaField("usaf", "STRING"), #station number, world metherorological org
        bigquery.SchemaField("wban", "STRING"), #wban number, weather bureau army
        bigquery.SchemaField("country", "STRING"),
        bigquery.SchemaField("lat", "FLOAT"),
        bigquery.SchemaField("lon", "FLOAT")
    ],
).to_dataframe()

dataframe_stations

/opt/anaconda3/lib/python3.8/site-packages/google/cloud/bigquery/client.py:444: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  warnings.warn(


,usaf,wban,country,lat,lon
0,007018,99999,None,0.000,0.000
1,007026,99999,AF,0.000,0.000
2,007070,99999,AF,0.000,0.000
3,008268,99999,AF,32.950,65.567
4,008307,99999,AF,0.000,0.000
...,...,...,...,...,...
29556,720282,03736,US,36.018,-75.671
29557,723070,13786,US,36.261,-76.175
29558,723629,53998,US,30.069,-93.804
29559,720499,00154,US,36.783,-76.450


In [627]:
dataframe_stations.to_csv('dataframe_stations.csv')

In [654]:
import urllib.parse
from urllib.request import urlopen
import json
import math

def getLocation(lat, lon):
    
    country=''
    country_ISO_CODE2=''
    
    if((not math.isnan(lat)) and (not math.isnan(lon))):
    
        lat_query=urllib.parse.quote(str(lat))
        lon_query=urllib.parse.quote(str(lon))

        url = "https://maps.googleapis.com/maps/api/geocode/json?key=&"
        url += "latlng=%s,%s&sensor=false" % (lat_query, lon_query)


        v = urlopen(url).read()
        j = json.loads(v)

        try:
            components = j['results'][0]['address_components']
            country = town = None
            for c in components:
                if "country" in c['types']:
                    country = c['long_name']
                    country_ISO_CODE2 = c['short_name']

        except IndexError:
            country=''
            country_ISO_CODE2=''
        
    return country_ISO_CODE2

In [815]:
getLocation(56.483, 15.517)

'SE'

In [816]:
getLocation(-18, 15)

'NA'

In [701]:
#Not being used
import geocoder
def getLocation2(lat,lon):
    
    code=''
    g = geocoder.osm([lat, lon], method='reverse')
    
    try:
        if "country_code" in g.json:
            code=g.json['country_code']      
    except IndexError:
            code=''

    return code

In [758]:
aux1=dataframe_stations

In [801]:
aux1.lat=aux1.lat.astype(int)
aux1.lon=aux1.lon.astype(int)
aux1=aux1.drop_duplicates(subset=['lat','lon'])

In [802]:
aux1

,usaf,wban,country,lat,lon
3,008268,99999,AF,32,65
5,010016,99999,NO,64,11
6,010017,99999,NO,59,2
7,010060,99999,NO,78,22
8,010071,99999,SV,78,15
...,...,...,...,...,...
29093,999999,63831,US,32,-89
29095,999999,63849,US,37,-86
29148,999999,96408,US,63,-150
29214,720375,54844,US,46,-83


In [805]:
#Retrieve country or each lat ang log values with int coordinates since country tag in NOAA GSOD dataset
#is not consistent and does not follow any convention in naming accesible
%%time
import time

cont=0
res=[]
ite=0
for index, row in aux1.iterrows():
    
    #Every 150 iterations we sleep the loop in order to avoid crashes in the API.
    if(cont == 150):
        print('Sleep:' + str(ite))
        ite=ite+1
        time.sleep(30)
        cont=0
        
    cont=cont+1
    res.append(getLocation(row['lat'], row['lon']))

Sleep:0
Sleep:1
Sleep:2
Sleep:3
Sleep:4
Sleep:5
Sleep:6
Sleep:7
Sleep:8
Sleep:9
Sleep:10
Sleep:11
Sleep:12
Sleep:13
Sleep:14
Sleep:15
Sleep:16
Sleep:17
Sleep:18
Sleep:19
Sleep:20
Sleep:21
Sleep:22
Sleep:23
Sleep:24
Sleep:25
Sleep:26
Sleep:27
Sleep:28
Sleep:29
Sleep:30
Sleep:31
Sleep:32
Sleep:33
Sleep:34
Sleep:35
Sleep:36
Sleep:37
Sleep:38
Sleep:39
Sleep:40
Sleep:41
Sleep:42
Sleep:43
Sleep:44
Sleep:45
Sleep:46
Sleep:47
Sleep:48
Sleep:49
Sleep:50
Sleep:51
Sleep:52
Sleep:53
Sleep:54
Sleep:55
Sleep:56
Sleep:57
Sleep:58
Sleep:59
Sleep:60
Sleep:61
Sleep:62
CPU times: user 3min 15s, sys: 11 s, total: 3min 26s
Wall time: 59min 7s


In [809]:
aux1['country_ISO_code_2']=res
aux1

<ipython-input-809-7cb49369c389>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aux1['country_ISO_code_2']=res


,usaf,wban,country,lat,lon,country_ISO_code_2
3,008268,99999,AF,32,65,AF
5,010016,99999,NO,64,11,NO
6,010017,99999,NO,59,2,
7,010060,99999,NO,78,22,SJ
8,010071,99999,SV,78,15,SJ
...,...,...,...,...,...,...
29093,999999,63831,US,32,-89,US
29095,999999,63849,US,37,-86,US
29148,999999,96408,US,63,-150,US
29214,720375,54844,US,46,-83,CA


In [810]:
aux1.to_csv('aux1_final.csv')

In [836]:
dataframe_stations_api = aux1[['usaf', 'wban', 'lat', 'lon', 'country_ISO_code_2']]
dataframe_stations_api.columns=['usaf', 'wban', 'lat', 'lon', 'country']
dataframe_stations_api['STN'] = dataframe_stations_api['usaf'] + '-' + dataframe_stations_api['wban']
dataframe_stations_api['LOC'] = dataframe_stations_api.lat.astype(str) + '|' + dataframe_stations_api.lon.astype(str)
dataframe_stations_api

,usaf,wban,lat,lon,country,STN,LOC
3,008268,99999,32,65,AF,008268-99999,32|65
5,010016,99999,64,11,NO,010016-99999,64|11
6,010017,99999,59,2,,010017-99999,59|2
7,010060,99999,78,22,SJ,010060-99999,78|22
8,010071,99999,78,15,SJ,010071-99999,78|15
...,...,...,...,...,...,...,...
29093,999999,63831,32,-89,US,999999-63831,32|-89
29095,999999,63849,37,-86,US,999999-63849,37|-86
29148,999999,96408,63,-150,US,999999-96408,63|-150
29214,720375,54844,46,-83,CA,720375-54844,46|-83


In [6]:
table1_gsod2020 = bigquery.TableReference.from_string(
    "bigquery-public-data.noaa_gsod.gsod2020"
)

dataframe_gsod2020= client.list_rows(table1_gsod2020,
    selected_fields=[
        bigquery.SchemaField("stn", "STRING"), #station number
        bigquery.SchemaField("wban", "STRING"), #station number
        bigquery.SchemaField("date", "DATE"),
        bigquery.SchemaField("temp", "FLOAT"), #mean temp of the day
        bigquery.SchemaField("prcp", "FLOAT"), #total_precipitation
    ],).to_dataframe()

dataframe_gsod2020

,stn,wban,date,temp,prcp
0,710420,99999,2020-01-07,44.3,0.00
1,703655,26552,2020-01-28,-43.2,0.00
2,702685,27518,2020-02-06,-46.8,0.00
3,716680,99999,2020-11-02,47.8,0.78
4,714480,99999,2020-06-17,81.7,0.00
...,...,...,...,...,...
4107416,956660,99999,2020-08-11,53.6,0.16
4107417,959660,99999,2020-08-30,49.4,0.08
4107418,999999,00458,2020-02-08,30.4,99.99
4107419,999999,00322,2020-10-17,31.3,99.99


In [7]:
table1_gsod2021 = bigquery.TableReference.from_string(
    "bigquery-public-data.noaa_gsod.gsod2021"
)

dataframe_gsod2021= client.list_rows(table1_gsod2021,
    selected_fields=[
        bigquery.SchemaField("stn", "STRING"), #station number
        bigquery.SchemaField("wban", "STRING"), #station number
        bigquery.SchemaField("date", "DATE"),
        bigquery.SchemaField("temp", "FLOAT"), #mean temp of the day
        bigquery.SchemaField("prcp", "FLOAT"), #total_precipitation
    ],).to_dataframe()

dataframe_gsod2021

/opt/anaconda3/lib/python3.8/site-packages/google/cloud/bigquery/client.py:444: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  warnings.warn(


,stn,wban,date,temp,prcp
0,010014,99999,2021-01-28,26.1,0.00
1,010014,99999,2021-02-01,25.2,0.00
2,010014,99999,2021-02-09,23.9,0.00
3,010014,99999,2021-03-19,41.9,0.00
4,010020,99999,2021-01-08,25.1,0.00
...,...,...,...,...,...
930672,945953,99999,2021-01-14,88.8,0.08
930673,943260,99999,2021-01-26,83.7,0.20
930674,943260,99999,2021-03-12,85.0,0.01
930675,945951,99999,2021-01-06,77.9,4.41


In [818]:
weather_data = pd.concat([dataframe_gsod2020, dataframe_gsod2021])
weather_data['STN'] = weather_data['stn'] + '-' + weather_data['wban']
weather_data

,stn,wban,date,temp,prcp,STN
0,710420,99999,2020-01-07,44.3,0.00,710420-99999
1,703655,26552,2020-01-28,-43.2,0.00,703655-26552
2,702685,27518,2020-02-06,-46.8,0.00,702685-27518
3,716680,99999,2020-11-02,47.8,0.78,716680-99999
4,714480,99999,2020-06-17,81.7,0.00,714480-99999
...,...,...,...,...,...,...
930672,945953,99999,2021-01-14,88.8,0.08,945953-99999
930673,943260,99999,2021-01-26,83.7,0.20,943260-99999
930674,943260,99999,2021-03-12,85.0,0.01,943260-99999
930675,945951,99999,2021-01-06,77.9,4.41,945951-99999


In [270]:
weather_data.to_csv('weather_data.csv')

In [277]:
weather_data.dtypes

stn      object
wban     object
date     object
temp    float64
prcp    float64
STN      object
dtype: object

In [822]:
dataframe_stations['STN'] = dataframe_stations['usaf'] + '-' + dataframe_stations['wban']
dataframe_stations

,usaf,wban,country,lat,lon,STN
0,007018,99999,None,0.000,0.000,007018-99999
1,007026,99999,AF,0.000,0.000,007026-99999
2,007070,99999,AF,0.000,0.000,007070-99999
3,008268,99999,AF,32.950,65.567,008268-99999
4,008307,99999,AF,0.000,0.000,008307-99999
...,...,...,...,...,...,...
29556,720282,03736,US,36.018,-75.671,720282-03736
29557,723070,13786,US,36.261,-76.175,723070-13786
29558,723629,53998,US,30.069,-93.804,723629-53998
29559,720499,00154,US,36.783,-76.450,720499-00154


In [834]:
weather_data_aux=pd.merge(weather_data, dataframe_stations,  how='left', left_on=['STN'], right_on = ['STN'])
weather_data_aux=weather_data_aux[['date', 'temp', 'prcp', 'lat', 'lon']]
weather_data_aux.lat=weather_data_aux.lat.fillna(0).astype(int)
weather_data_aux.lon=weather_data_aux.lon.fillna(0).astype(int)
weather_data_aux.lat.astype(int)
weather_data_aux.lon.astype(int)
weather_data_aux['LOC'] = weather_data_aux['lat'].astype(str) + '|' + weather_data_aux['lon'].astype(str)
weather_data_aux

,date,temp,prcp,lat,lon,LOC
0,2020-01-07,44.3,0.00,49,-123,49|-123
1,2020-01-28,-43.2,0.00,65,-156,65|-156
2,2020-02-06,-46.8,0.00,70,-157,70|-157
3,2020-11-02,47.8,0.78,45,-66,45|-66
4,2020-06-17,81.7,0.00,50,-96,50|-96
...,...,...,...,...,...,...
5038093,2021-01-14,88.8,0.08,-17,123,-17|123
5038094,2021-01-26,83.7,0.20,-23,133,-23|133
5038095,2021-03-12,85.0,0.01,-23,133,-23|133
5038096,2021-01-06,77.9,4.41,-27,152,-27|152


In [837]:
dataframe_stations_api

,usaf,wban,lat,lon,country,STN,LOC
3,008268,99999,32,65,AF,008268-99999,32|65
5,010016,99999,64,11,NO,010016-99999,64|11
6,010017,99999,59,2,,010017-99999,59|2
7,010060,99999,78,22,SJ,010060-99999,78|22
8,010071,99999,78,15,SJ,010071-99999,78|15
...,...,...,...,...,...,...,...
29093,999999,63831,32,-89,US,999999-63831,32|-89
29095,999999,63849,37,-86,US,999999-63849,37|-86
29148,999999,96408,63,-150,US,999999-96408,63|-150
29214,720375,54844,46,-83,CA,720375-54844,46|-83


In [845]:
weather_final_df=pd.merge(weather_data_aux, dataframe_stations_api,  how='left', left_on=['LOC'], right_on = ['LOC'])
weather_final_df=weather_final_df[['country', 'date', 'temp', 'prcp']]

#null values in dataframe come with value 9999.9 or 999.9, converted to null instead
weather_final_df['temp'] = weather_final_df['temp'].apply(lambda x: np.nan if x==9999.9 else x)
weather_final_df['prcp'] = weather_final_df['prcp'].apply(lambda x: np.nan if x==99.99 else x)
# convert temp into celsius
weather_final_df['temp'] = (weather_final_df['temp'] - 32) / 1.8
#date column to DatetimeIndex for later grouping by day
weather_final_df['date'] = pd.to_datetime(weather_final_df['date'])

weather_final_df

,country,date,temp,prcp
0,US,2020-01-07,6.833333,0.00
1,US,2020-01-28,-41.777778,0.00
2,US,2020-02-06,-43.777778,0.00
3,CA,2020-11-02,8.777778,0.78
4,CA,2020-06-17,27.611111,0.00
...,...,...,...,...
5038093,AU,2021-01-14,31.555556,0.08
5038094,AU,2021-01-26,28.722222,0.20
5038095,AU,2021-03-12,29.444444,0.01
5038096,AU,2021-01-06,25.500000,4.41


In [846]:
weather_final_df[weather_final_df['country']=='ES']

,country,date,temp,prcp
165,ES,2020-01-13,-0.722222,NaN
589,ES,2020-01-03,3.722222,0.00
2553,ES,2020-01-15,17.111111,0.00
4426,ES,2020-01-16,17.777778,0.00
4475,ES,2020-08-05,23.000000,0.00
...,...,...,...,...
5037350,ES,2021-01-04,4.500000,1.65
5037351,ES,2021-02-05,7.777778,0.65
5037884,ES,2021-01-04,3.277778,1.67
5038057,ES,2021-01-01,5.944444,0.64


In [868]:
#Average temperature in Germany January
weather_final_df[(weather_final_df['country']=='DE') & (weather_final_df['date'].dt.month==1)].temp.mean()

2.377678902484235

In [869]:
#Average temperature in Spain March
weather_final_df[(weather_final_df['country']=='ES') & (weather_final_df['date'].dt.month==4)].temp.mean()

13.515903801396423

In [879]:
#Average precipitation in Spain February
weather_final_df[(weather_final_df['country']=='ES') & (weather_final_df['date'].dt.month==2)].prcp.mean()

0.05671347171347152

In [881]:
#Average precipitation in Great Britain February
weather_final_df[(weather_final_df['country']=='GB') & (weather_final_df['date'].dt.month==2)].prcp.mean()

0.1622366065280232

In [882]:
weather_final_df.to_csv('weather_final_df.csv')

In [402]:
#########################################################################################################